# NLMD
https://scikit-image.org/docs/stable/auto_examples/filters/plot_nonlocal_means.html#sphx-glr-auto-examples-filters-plot-nonlocal-means-py

## Denoise image

Import Library

In [ ]:
import os
import numpy as np
import pydicom

import numpy as np
import matplotlib.pyplot as plt
from skimage import img_as_float, exposure
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio
from skimage.util import random_noise
import os


import pydicom
import skimage
print(skimage.__version__)

import cv2

In [ ]:
noisy = pydicom.dcmread("C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data/Added Noise/CATHLAB/Poisson/CATHLAB Noisy (Poisson 0.70)/FCAVDP0G.dcm")
normal = pydicom.dcmread("C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data/Original/CATHLAB/FCAVDP0G")

In [ ]:
# Check dimensions
rows = normal.Rows
columns = normal.Columns

# For multi-slice or 3D images:
if hasattr(normal, 'NumberOfFrames'):
    frames = normal.NumberOfFrames
    print(f"Dimensions: {frames} slices, {rows} x {columns} pixels per slice")
else:
    print(f"Dimensions: {rows} x {columns} pixels")

# Access the pixel data as a NumPy array
import numpy as np
pixel_array = normal.pixel_array
print(f"Pixel array shape: {pixel_array.shape}")

# Get the pixel array from the DICOM file and check its shape and type
dicom_pixel_array = normal.pixel_array
dicom_pixel_shape = dicom_pixel_array.shape
dicom_pixel_dtype =  dicom_pixel_array.dtype

dicom_pixel_shape, dicom_pixel_dtype


print(f"Shape of DICOM pixel array: {dicom_pixel_shape}")
print(f"Data type of DICOM pixel array: {dicom_pixel_dtype}")
print(pixel_array.ndim)

In [ ]:
pixel_array = normal.pixel_array
print(f"Pixel array shape: {pixel_array.shape}")
print(f"Number of dimensions: {pixel_array.ndim}")

# Ensure the array is 2D
pixel_array = np.squeeze(pixel_array)
print(f"Pixel array shape after squeeze: {pixel_array.shape}")

# Convert to float format
pixel_array_float = img_as_float(pixel_array)

In [ ]:
pixel_array_noisy = noisy.pixel_array
print(f"Pixel array shape: {pixel_array.shape}")
print(f"Number of dimensions: {pixel_array.ndim}")

# Ensure the array is 2D
pixel_array = np.squeeze(pixel_array_noisy)
print(f"Pixel array shape after squeeze: {pixel_array.shape}")

# Convert to float format
pixel_array_float_noisy = img_as_float(pixel_array_noisy)

In [ ]:
# Define patch keywords
patch_kw = dict(
    patch_size=2, patch_distance=1
)

In [ ]:
# Apply Non-Local Means Denoising
denoised_image_noisy = denoise_nl_means(pixel_array_float_noisy, h=0.1, fast_mode=False, **patch_kw)


In [ ]:
psnr = peak_signal_noise_ratio(pixel_array_float_noisy, pixel_array_float)
print(f'PSNR = {psnr:0.2f} dB')

psnr = peak_signal_noise_ratio(denoised_image_noisy, pixel_array_float)
print(f'PSNR = {psnr:0.2f} dB')

In [ ]:
# Plot the original and denoised images
plt.figure(figsize=(10, 5))

# Original image
plt.subplot(1, 3, 1)
plt.imshow(pixel_array_float, cmap='gray')
plt.title("Original Image")
plt.axis("off")

# Noisy image
plt.subplot(1, 3, 2)
plt.imshow(pixel_array_float_noisy, cmap='gray')
plt.title("Noisy Image")
plt.axis("off")

# Denoised image
plt.subplot(1, 3, 3)
plt.imshow(denoised_image_noisy, cmap='gray')
plt.title("Denoised Image")
plt.axis("off")


plt.tight_layout()
plt.show()

In [ ]:

# Get original pixel range
min_val = noisy.pixel_array.min()
max_val = noisy.pixel_array.max()

# Rescale denoised image to original intensity range
denoised_rescaled = (denoised_image_noisy - denoised_image_noisy.min()) / \
                    (denoised_image_noisy.max() - denoised_image_noisy.min())
denoised_rescaled = denoised_rescaled * (max_val - min_val) + min_val

# Convert to uint16
noisy.PixelData = denoised_rescaled.astype('uint16').tobytes()
noisy.save_as('HASIL_CATHLAB_NLMD(0.29).dcm')


## Process NLMD single file

In [ ]:
import os
import numpy as np
import pydicom

import numpy as np
import matplotlib.pyplot as plt
from skimage import img_as_float, exposure
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio
from skimage.util import random_noise
import os


import pydicom
import skimage
print(skimage.__version__)

import cv2

In [ ]:
def denoise_image(image):
    """
    Apply Non-Local Means Denoising to a given image.
    """
    # Convert to float format
    image_float = img_as_float(image)
    
    
    # Set patch keyword arguments
    patch_kw = dict(
        patch_size = 2,  # Patch size for denoising
        patch_distance = 1  # Search window size
    )
    
    # Apply Non-Local Means Denoising
    denoised_image = denoise_nl_means(image_float, fast_mode=False, **patch_kw)
    
    return denoised_image

def process_dicom_images(input_dir, output_dir):
    """
    Process all DICOM files in the input directory and its subdirectories,
    apply Non-Local Means Denoising, and save the denoised images to the output directory.
    """
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Walk through all subdirectories and files
    for root, _, files in os.walk(input_dir):
        for filename in files:
            file_path = os.path.join(root, filename)
            if os.path.isfile(file_path):
                try:
                    # Read the DICOM file
                    dicom = pydicom.dcmread(file_path)
                    image = dicom.pixel_array
                    
                    # Apply Non-Local Means Denoising
                    denoised_image = denoise_image(image)
                    
                    # Prepare the output path (preserve subdirectory structure)
                    relative_path = os.path.relpath(root, input_dir)
                    sub_output_dir = os.path.join(output_dir, relative_path)
                    if not os.path.exists(sub_output_dir):
                        os.makedirs(sub_output_dir)
                    output_path = os.path.join(sub_output_dir, filename)
                    
                    # Save the denoised image as a new DICOM file
                    dicom.PixelData = (denoised_image * np.max(image)).astype(np.uint16).tobytes()
                    dicom.save_as(output_path)
                    print(f"Processed: {file_path} -> {output_path}")
                except Exception as e:
                    print(f"Skipping {file_path}: Not a valid DICOM file ({e})")

# Example usage
input_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data/Added Noise/CATHLAB/Poisson/CATHLAB Noisy (Poisson 0.90)"
output_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Results/NLMD/Poisson/CATHLAB Noisy (Poisson 0.90)"
process_dicom_images(input_directory, output_directory)


## Process NLMD Multiple File

In [1]:
import os
import numpy as np
import pydicom

import numpy as np
import matplotlib.pyplot as plt
from skimage import img_as_float, exposure
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio
from skimage.util import random_noise
import os


import pydicom
import skimage
print(skimage.__version__)

import cv2

0.21.0


In [2]:
import time
import psutil
import tracemalloc
import pynvml
import tensorflow as tf
import threading

# Initialize GPU monitoring
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

# Track CPU utilization (system-wide) over time
cpu_utilization_samples = []
stop_sampling = False

def sample_cpu_utilization(interval=1.0):
    while not stop_sampling:
        cpu_utilization_samples.append(psutil.cpu_percent(interval=interval))

# Start CPU utilization sampling in the background
sampling_thread = threading.Thread(target=sample_cpu_utilization)
sampling_thread.start()

# Start resource tracking
start_time = time.time()
tracemalloc.start()
process = psutil.Process()
cpu_times_start = process.cpu_times()

In [3]:
def denoise_image(image):
    """
    Apply Non-Local Means Denoising to a given image.
    """
    # Convert to float format
    image_float = img_as_float(image)
    
    
    # Set patch keyword arguments
    patch_kw = dict(
        patch_size = 5,  # Patch size for denoising
        patch_distance= 7  # Search window size
    )
    
    # Apply Non-Local Means Denoising
    denoised_image = denoise_nl_means(image_float, fast_mode=False, **patch_kw)
    
    return denoised_image

def process_dicom_images(input_dir, output_dir):
    """
    Process all DICOM files in the input directory and its subdirectories,
    apply Non-Local Means Denoising, and save the denoised images to the output directory.
    """
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Walk through all subdirectories and files
    for root, _, files in os.walk(input_dir):
        for filename in files:
            file_path = os.path.join(root, filename)
            if os.path.isfile(file_path):
                try:
                    # Read the DICOM file
                    dicom = pydicom.dcmread(file_path)
                    image = dicom.pixel_array
                    
                    # Get original pixel range
                    min_val = image.min()
                    max_val = image.max()
                    
                    # Apply Non-Local Means Denoising
                    denoised_image = denoise_image(image)
                    
                    # Rescale denoised image to original intensity range
                    denoised_rescaled = (denoised_image - denoised_image.min()) / \
                                        (denoised_image.max() - denoised_image.min())
                    denoised_rescaled = denoised_rescaled * (max_val - min_val) + min_val
                    
                    # Convert to uint16
                    dicom.PixelData = denoised_rescaled.astype('uint16').tobytes()
                    
                    # Prepare the output path (preserve subdirectory structure)
                    relative_path = os.path.relpath(root, input_dir)
                    sub_output_dir = os.path.join(output_dir, relative_path)
                    if not os.path.exists(sub_output_dir):
                        os.makedirs(sub_output_dir)
                    output_path = os.path.join(sub_output_dir, filename)
                    
                    dicom.save_as(output_path)
                    print(f"Processed: {file_path} -> {output_path}")
                except Exception as e:
                    print(f"Skipping {file_path}: Not a valid DICOM file ({e})")

# Example usage
input_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/Data 10/Original/CATHLAB"
output_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (4 apr)/Results 10/NLMD57/Original"
process_dicom_images(input_directory, output_directory)

Processed: C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/Data 10/Original/CATHLAB\FCAVDP00.dcm -> C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (4 apr)/Results 10/NLMD57/Original\.\FCAVDP00.dcm
Processed: C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/Data 10/Original/CATHLAB\FCAVDP0A.dcm -> C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (4 apr)/Results 10/NLMD57/Original\.\FCAVDP0A.dcm
Processed: C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/Data 10/Original/CATHLAB\FCAVDP0C.dcm -> C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (4 apr)/Results 10/NLMD57/Original\.\FCAVDP0C.dcm
Processed: C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/Data 10/Original/CATHLAB\FCAVDP0E.dcm -> C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (4 apr)/Results 10/NLMD57/Original\.\FCAVDP0E.dcm
Processed: C:/Matthe

In [4]:
# End resource tracking
cpu_times_end = process.cpu_times()
end_time = time.time()
stop_sampling = True
sampling_thread.join()

# RAM tracking
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# GPU metrics
mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
utilization = pynvml.nvmlDeviceGetUtilizationRates(handle)
pynvml.nvmlShutdown()

gpu_mem_used = mem_info.used / 1024**2  # MB
gpu_util_percent = utilization.gpu      # % GPU utilization

# CPU usage calculation
execution_time = end_time - start_time
cpu_time_used = (cpu_times_end.user + cpu_times_end.system) - (cpu_times_start.user + cpu_times_start.system)
cpu_usage_percent = (cpu_time_used / execution_time) * 100 if execution_time > 0 else 0

# Average system-wide CPU utilization
avg_cpu_utilization = sum(cpu_utilization_samples) / len(cpu_utilization_samples) if cpu_utilization_samples else 0

# RAM usage
ram_usage = peak / 10**6  # Convert to MB

# Print the results
print(f"Execution Time: {execution_time:.2f} seconds")
print(f"CPU Usage (Script): {cpu_usage_percent:.2f}%")
print(f"CPU Utilization (System Avg): {avg_cpu_utilization:.2f}%")
print(f"Peak RAM Usage: {ram_usage:.2f} MB")
print(f"GPU Memory Used: {gpu_mem_used:.2f} MB")
print(f"GPU Utilization: {gpu_util_percent:.2f}%")
# Cleanup
pynvml.nvmlShutdown()

Execution Time: 22.30 seconds
CPU Usage (Script): 97.90%
CPU Utilization (System Avg): 7.26%
Peak RAM Usage: 3.86 MB
GPU Memory Used: 775.68 MB
GPU Utilization: 2.00%


## Pengolahan Variasi NLMD

In [ ]:
import os
import numpy as np
import pydicom
import pandas as pd
from skimage.restoration import denoise_nl_means
from skimage import img_as_float
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
def denoise_image(image, patch_size, patch_distance):
    """
    Apply Non-Local Means Denoising to a given image with specified patch size and distance.
    """
    image_float = img_as_float(image)
    
    patch_kw = dict(
        patch_size=patch_size, 
        patch_distance=patch_distance,
        fast_mode=False
    )
    return denoise_nl_means(image_float, **patch_kw)

def compute_psnr(denoised, original):
    """Compute PSNR between original and denoised images."""
    return psnr(denoised, original, data_range=original.max() - original.min())

In [ ]:
def process_images(noise_levels, patch_sizes, patch_distances, noisy_dir, normal_dir, output_xlsx):
    """
    Process DICOM images for various noise levels, patch sizes, and patch distances.
    Saves the PSNR values into an Excel file.
    """
    results = []
    
    for noise in noise_levels:
        noisy_subdir = os.path.join(noisy_dir, f"CATHLAB Noisy ({noise:.2f})")
        
        for psize in patch_sizes:
            for pdistance in patch_distances:
                psnr_values = []
                
                for filename in os.listdir(noisy_subdir):
                    noisy_path = os.path.join(noisy_subdir, filename)
                    normal_path = os.path.join(normal_dir, filename)
                    
                    if os.path.isfile(noisy_path) and os.path.isfile(normal_path):
                        try:
                            noisy_image = pydicom.dcmread(noisy_path)
                            noisy_image = noisy_image.pixel_array

                            normal_image = pydicom.dcmread(normal_path)
                            normal_image = normal_image.pixel_array
                            
                            denoised_image = denoise_image(noisy_image, psize, pdistance)
                            
                            min_val, max_val = noisy_image.min(), noisy_image.max()
                            
                            denoised_rescaled = (denoised_image - denoised_image.min()) / \
                            (denoised_image.max() - denoised_image.min())
                            denoised_rescaled = denoised_rescaled * (max_val - min_val) + min_val
                            
                            psnr_value = compute_psnr(denoised_rescaled, normal_image)
                            psnr_values.append(psnr_value)
                        except Exception as e:
                            print(f"Skipping {filename}: {e}")
                
                avg_psnr = np.mean(psnr_values) if psnr_values else None
                results.append([noise, psize, pdistance, avg_psnr])
    
    df = pd.DataFrame(results, columns=["Noise Level", "Patch Size", "Patch Distance", "PSNR"])
    df.to_excel(output_xlsx, index=False)
    print(f"Results saved to {output_xlsx}")

In [ ]:
# Define parameters
noise_levels = np.arange(0.10, 0.30, 0.01)
patch_sizes = [11, 16, 32, 64]
patch_distances = [1, 3, 4, 5, 7, 9, 11, 13, 15,21,35]

noisy_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (28 feb)/Data10/Added Noise/CATHLAB/Gaussian"
normal_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (28 feb)/Data10/Original/CATHLAB"
output_excel = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/NLMD_Patch_PSNR_Gaussian_64.xlsx"

In [ ]:
# Run the process
process_images(noise_levels, patch_sizes, patch_distances, noisy_directory, normal_directory, output_excel)

## Plot image

In [ ]:
noisy_NLMD = pydicom.dcmread("C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data/Added Noise/CATHLAB/Gaussian/CATHLAB Noisy (0.30)/FCAVDP8K.dcm")
denoised_NLMD = pydicom.dcmread("C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Results/NLMD/Gaussian/CATHLAB Noisy (0.30)/FCAVDP8K.dcm")
normal_NLMD = pydicom.dcmread("C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data/Original/CATHLAB/FCAVDP8K.dcm")

In [ ]:
pixel_array = normal_NLMD.pixel_array
print(f"Pixel array shape: {pixel_array.shape}")
print(f"Number of dimensions: {pixel_array.ndim}")

# Ensure the array is 2D
pixel_array = np.squeeze(pixel_array)
print(f"Pixel array shape after squeeze: {pixel_array.shape}")

# Convert to float format
pixel_array_float_cnn = img_as_float(pixel_array)

In [ ]:
pixel_array = noisy_NLMD.pixel_array
print(f"Pixel array shape: {pixel_array.shape}")
print(f"Number of dimensions: {pixel_array.ndim}")

# Ensure the array is 2D
pixel_array = np.squeeze(pixel_array)
print(f"Pixel array shape after squeeze: {pixel_array.shape}")

# Convert to float format
pixel_array_float_noisy_cnn = img_as_float(pixel_array)

In [ ]:
pixel_array = denoised_NLMD.pixel_array
print(f"Pixel array shape: {pixel_array.shape}")
print(f"Number of dimensions: {pixel_array.ndim}")

# Ensure the array is 2D
pixel_array = np.squeeze(pixel_array)
print(f"Pixel array shape after squeeze: {pixel_array.shape}")

# Convert to float format
pixel_array_float_denoised_cnn = img_as_float(pixel_array)

In [ ]:
psnr = peak_signal_noise_ratio(pixel_array_float_noisy_cnn, pixel_array_float_cnn)
print(f'PSNR = {psnr:0.2f} dB')

psnr = peak_signal_noise_ratio(pixel_array_float_denoised_cnn, pixel_array_float_cnn)
print(f'PSNR = {psnr:0.2f} dB')

In [ ]:
# Plot the original and denoised images
plt.figure(figsize=(10, 5))

# Original image
plt.subplot(1, 3, 1)
plt.imshow(pixel_array_float_cnn, cmap='gray')
plt.title("Original Image")
plt.axis("off")

# Noisy image
plt.subplot(1, 3, 2)
plt.imshow(pixel_array_float_noisy_cnn, cmap='gray')
plt.title("Noisy Image")
plt.axis("off")

# Denoised image
plt.subplot(1, 3, 3)
plt.imshow(pixel_array_float_denoised_cnn, cmap='gray')
plt.title("Denoised Image")
plt.axis("off")


plt.tight_layout()
plt.show()

## Process file

In [ ]:
import os

# Define the path to your original DICOM folder
original_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data/Original/CATHLAB"

# Iterate through files in the directory
for filename in os.listdir(original_dir):
    file_path = os.path.join(original_dir, filename)

    # Check if it's a file and doesn't already have an extension
    if os.path.isfile(file_path) and '.' not in filename:
        new_filename = filename + ".dcm"
        new_file_path = os.path.join(original_dir, new_filename)

        # Rename the file
        os.rename(file_path, new_file_path)
        print(f"Renamed: {filename} -> {new_filename}")

print("Renaming complete.")


In [ ]:
import os

# Define the main directory and the list of files to keep
original_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data10"
files_to_keep = {
    "FCAVDP00", "FCAVDP0A", "FCAVDP0C", "FCAVDP0E", "FCAVDP0G",
    "FCAVDP0H", "FCAVDP0K", "FCAVDP0M", "FCAVDP0O", "FCAVDP0Q"
}

# Walk through all subdirectories and files
for root, _, files in os.walk(original_dir):
    for filename in files:
        file_path = os.path.join(root, filename)
        
        # Extract the base filename (ignoring extensions)
        base_name, ext = os.path.splitext(filename)

        # Delete files that are not in the keep list
        if base_name not in files_to_keep:
            os.remove(file_path)
            print(f"Deleted: {file_path}")

print("Cleanup complete.")


## PSNR Analisis Sementara

In [ ]:
import os
import pydicom
import numpy as np
from skimage.metrics import peak_signal_noise_ratio
from skimage import img_as_float

# Define paths


noisy_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data10/Added Noise/CATHLAB/Gaussian/CATHLAB Noisy (0.30)"
denoised_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Results 10/NLMD/Gaussian/CATHLAB Noisy (0.30)"
original_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data10/Original/CATHLAB"
# Get all file names from the original directory
dicom_files = [f for f in os.listdir(original_dir) if f.endswith(".dcm")]

# Store results
psnr_results = []

# Iterate through each file and compute PSNR
for filename in dicom_files:
    # Load corresponding images
    normal_path = os.path.join(original_dir, filename)
    noisy_path = os.path.join(noisy_dir, filename)
    denoised_path = os.path.join(denoised_dir, filename)

    # Ensure all files exist
    if not (os.path.exists(noisy_path) and os.path.exists(denoised_path)):
        print(f"Skipping {filename}: Corresponding noisy/denoised file not found.")
        continue

    normal_dcm = pydicom.dcmread(normal_path)
    noisy_dcm = pydicom.dcmread(noisy_path)
    denoised_dcm = pydicom.dcmread(denoised_path)

    # Convert pixel data to float format
    normal_img = img_as_float(np.squeeze(normal_dcm.pixel_array))
    noisy_img = img_as_float(np.squeeze(noisy_dcm.pixel_array))
    denoised_img = img_as_float(np.squeeze(denoised_dcm.pixel_array))

    # Compute PSNR
    psnr_noisy = peak_signal_noise_ratio(noisy_img, normal_img)
    psnr_denoised = peak_signal_noise_ratio(denoised_img, normal_img)

 

In [ ]:
# Print the final summary
print("\nFinal PSNR Results:")
print(f"{'Filename':<10} {'PSNR (Noisy)':<15} {'PSNR (Denoised)':<15}")
print("=" * 40)
for filename, psnr_noisy, psnr_denoised in psnr_results:
    print(f"{filename:<10} {psnr_noisy:<15.2f} {psnr_denoised:<15.2f}")

print("\nProcessing complete.")

In [ ]:
import os
import pydicom
import numpy as np
from skimage.metrics import peak_signal_noise_ratio
from skimage import img_as_float

# Define paths
original_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data10/Original/CATHLAB"
noisy_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data10/Added Noise/CATHLAB/Poisson/CATHLAB Noisy (0.79)"
denoised_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Results 10/RED-CNN/Poisson/CATHLAB Noisy (0.79)"

# List of files to process
files_to_process = [
    "FCAVDP00", "FCAVDP0A", "FCAVDP0C", "FCAVDP0E", "FCAVDP0G",
    "FCAVDP0H", "FCAVDP0K", "FCAVDP0M", "FCAVDP0O", "FCAVDP0Q"
]

# Store results
psnr_results = []
psnr_noisy_list = []
psnr_denoised_list = []

# Iterate through each file and compute PSNR
for filename in files_to_process:
    normal_path = os.path.join(original_dir, filename + ".dcm")
    noisy_path = os.path.join(noisy_dir, filename + ".dcm")
    denoised_path = os.path.join(denoised_dir, filename + ".dcm")

    # Ensure all files exist
    if not (os.path.exists(normal_path) and os.path.exists(noisy_path) and os.path.exists(denoised_path)):
        print(f"Skipping {filename}: Missing file(s).")
        continue

    # Load DICOM images
    normal_dcm = pydicom.dcmread(normal_path)
    noisy_dcm = pydicom.dcmread(noisy_path)
    denoised_dcm = pydicom.dcmread(denoised_path)

    # Convert pixel data to float format
    normal_img = img_as_float(np.squeeze(normal_dcm.pixel_array))
    noisy_img = img_as_float(np.squeeze(noisy_dcm.pixel_array))
    denoised_img = img_as_float(np.squeeze(denoised_dcm.pixel_array))

    # Compute PSNR
    psnr_noisy = peak_signal_noise_ratio(normal_img, noisy_img)
    psnr_denoised = peak_signal_noise_ratio(normal_img, denoised_img)

    # Store the result
    psnr_results.append((filename, psnr_noisy, psnr_denoised))
    psnr_noisy_list.append(psnr_noisy)
    psnr_denoised_list.append(psnr_denoised)

# Compute mean and standard deviation
mean_psnr_noisy = np.mean(psnr_noisy_list)
std_psnr_noisy = np.std(psnr_noisy_list)
mean_psnr_denoised = np.mean(psnr_denoised_list)
std_psnr_denoised = np.std(psnr_denoised_list)

# Print the final summary
print("\nFinal PSNR Results:")
print(f"{'Filename':<10} {'PSNR (Noisy)':<15} {'PSNR (Denoised)':<15}")
print("=" * 40)
for filename, psnr_noisy, psnr_denoised in psnr_results:
    print(f"{filename:<10} {psnr_noisy:<15.2f} {psnr_denoised:<15.2f}")

# Print mean and standard deviation
print("=" * 40)
print(f"{'Mean':<10} {mean_psnr_noisy:<15.2f} {mean_psnr_denoised:<15.2f}")
print(f"{'Std Dev':<10} {std_psnr_noisy:<15.2f} {std_psnr_denoised:<15.2f}")
print("\nProcessing complete.")


In [ ]:
import os
import pydicom
import numpy as np
from skimage.metrics import peak_signal_noise_ratio
from skimage import img_as_float

# Define base paths

# Define paths
base_original_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data10/Original/CATHLAB"
base_noisy_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data10/Added Noise/CATHLAB/Gaussian"
base_denoised_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Results 10/RED-CNN/Gaussian"

# Noise levels to process (0.10 to 0.30 in increments of 0.01)
noise_levels = [f"{x:.2f}" for x in np.arange(0.10, 0.31, 0.01)]

# List of filenames to process
files_to_process = [
    "FCAVDP00", "FCAVDP0A", "FCAVDP0C", "FCAVDP0E", "FCAVDP0G",
    "FCAVDP0H", "FCAVDP0K", "FCAVDP0M", "FCAVDP0O", "FCAVDP0Q"
]

# Dictionary to store PSNR results per noise level
psnr_summary = {}

# Process each noise level
for noise_level in noise_levels:
    print(f"\nProcessing noise level: {noise_level}...")

    noisy_folder = os.path.join(base_noisy_dir, f"CATHLAB Noisy ({noise_level})")
    denoised_folder = os.path.join(base_denoised_dir, f"CATHLAB Noisy ({noise_level})")

    psnr_results = []
    psnr_noisy_list = []
    psnr_denoised_list = []

    for filename in files_to_process:
        normal_path = os.path.join(base_original_dir, filename + ".dcm")
        noisy_path = os.path.join(noisy_folder, filename + ".dcm")
        denoised_path = os.path.join(denoised_folder, filename + ".dcm")

        # Ensure all files exist
        if not (os.path.exists(normal_path) and os.path.exists(noisy_path) and os.path.exists(denoised_path)):
            print(f"Skipping {filename} for noise level {noise_level}: Missing file(s).")
            continue

        # Load DICOM images
        normal_dcm = pydicom.dcmread(normal_path)
        noisy_dcm = pydicom.dcmread(noisy_path)
        denoised_dcm = pydicom.dcmread(denoised_path)

        # Convert pixel data to float format
        normal_img = img_as_float(np.squeeze(normal_dcm.pixel_array))
        noisy_img = img_as_float(np.squeeze(noisy_dcm.pixel_array))
        denoised_img = img_as_float(np.squeeze(denoised_dcm.pixel_array))

        # Compute PSNR
        psnr_noisy = peak_signal_noise_ratio(normal_img, noisy_img)
        psnr_denoised = peak_signal_noise_ratio(normal_img, denoised_img)

        # Store the results
        psnr_results.append((filename, psnr_noisy, psnr_denoised))
        psnr_noisy_list.append(psnr_noisy)
        psnr_denoised_list.append(psnr_denoised)

    # Compute mean and standard deviation
    if psnr_noisy_list and psnr_denoised_list:
        mean_psnr_noisy = np.mean(psnr_noisy_list)
        std_psnr_noisy = np.std(psnr_noisy_list)
        mean_psnr_denoised = np.mean(psnr_denoised_list)
        std_psnr_denoised = np.std(psnr_denoised_list)

        # Store results in dictionary
        psnr_summary[noise_level] = {
            "mean_noisy": mean_psnr_noisy,
            "std_noisy": std_psnr_noisy,
            "mean_denoised": mean_psnr_denoised,
            "std_denoised": std_psnr_denoised,
            "results": psnr_results
        }

    # Print results for the current noise level
    print("\nPSNR Results:")
    print(f"{'Filename':<10} {'PSNR (Noisy)':<15} {'PSNR (Denoised)':<15}")
    print("=" * 40)
    for filename, psnr_noisy, psnr_denoised in psnr_results:
        print(f"{filename:<10} {psnr_noisy:<15.2f} {psnr_denoised:<15.2f}")

    # Print mean and standard deviation
    print("=" * 40)
    print(f"{'Mean':<10} {mean_psnr_noisy:<15.2f} {mean_psnr_denoised:<15.2f}")
    print(f"{'Std Dev':<10} {std_psnr_noisy:<15.2f} {std_psnr_denoised:<15.2f}")

# Final summary across all noise levels
print("\nFinal Summary of All Noise Levels:")
print(f"{'Noise Level':<10} {'Mean PSNR (Noisy)':<20} {'Mean PSNR (Denoised)':<20}")
print("=" * 60)
for noise_level, values in psnr_summary.items():
    print(f"{noise_level:<10} {values['mean_noisy']:<20.2f} {values['mean_denoised']:<20.2f}")

print("\nProcessing complete.")

In [ ]:
# Save results to a text file
output_file = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/PSNR_Results.txt"
with open(output_file, "w") as f:
    f.write("Final Summary of All Noise Levels:\n")
    f.write(f"{'Noise Level':<10} {'Mean PSNR (Noisy)':<20} {'Mean PSNR (Denoised)':<20}\n")
    f.write("=" * 60 + "\n")
    for noise_level, values in psnr_summary.items():
        f.write(f"{noise_level:<10} {values['mean_noisy']:<20.2f} {values['mean_denoised']:<20.2f}\n")
    
    f.write("\nDetailed Results for Each Noise Level:\n")
    for noise_level, values in psnr_summary.items():
        f.write(f"\nNoise Level: {noise_level}\n")
        f.write(f"{'Filename':<10} {'PSNR (Noisy)':<15} {'PSNR (Denoised)':<15}\n")
        f.write("=" * 40 + "\n")
        for filename, psnr_noisy, psnr_denoised in values["results"]:
            f.write(f"{filename:<10} {psnr_noisy:<15.2f} {psnr_denoised:<15.2f}\n")
        f.write("=" * 40 + "\n")
        f.write(f"{'Mean':<10} {values['mean_noisy']:<15.2f} {values['mean_denoised']:<15.2f}\n")
        f.write(f"{'Std Dev':<10} {values['std_noisy']:<15.2f} {values['std_denoised']:<15.2f}\n")

print(f"\nResults saved to {output_file}")

In [ ]:
import matplotlib.pyplot as plt

# Extract data for plotting
noise_levels = sorted(psnr_summary.keys())  # Ensure noise levels are in ascending order
mean_psnr_noisy = [psnr_summary[nl]["mean_noisy"] for nl in noise_levels]
mean_psnr_denoised = [psnr_summary[nl]["mean_denoised"] for nl in noise_levels]

# Plot PSNR values
plt.figure(figsize=(10, 6))
plt.plot(noise_levels, mean_psnr_noisy, marker='o', linestyle='-', color='r', label='Noisy Images')
plt.plot(noise_levels, mean_psnr_denoised, marker='s', linestyle='-', color='b', label='Denoised Images')

# Labels and title
plt.xlabel("Gaussian Noise Standard Deviation")
plt.ylabel("Mean PSNR (dB)")
plt.title("PSNR Analysis of Noisy vs. Denoised Images")
plt.legend()
plt.grid(True)

In [ ]:
import os
import pydicom
import numpy as np
from skimage.metrics import peak_signal_noise_ratio
from skimage import img_as_float

# Define base paths

# Define paths
base_original_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data10/Original/CATHLAB"
base_noisy_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data10/Added Noise/CATHLAB/Poisson"
base_denoised_dir = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Results 10/RED-CNN/Poisson"

# Noise levels to process (0.10 to 0.30 in increments of 0.01)
noise_levels = [f"{x:.2f}" for x in np.arange(0.70, 0.91, 0.01)]

# List of filenames to process
files_to_process = [
    "FCAVDP00", "FCAVDP0A", "FCAVDP0C", "FCAVDP0E", "FCAVDP0G",
    "FCAVDP0H", "FCAVDP0K", "FCAVDP0M", "FCAVDP0O", "FCAVDP0Q"
]

# Dictionary to store PSNR results per noise level
psnr_summary = {}

# Process each noise level
for noise_level in noise_levels:
    print(f"\nProcessing noise level: {noise_level}...")

    noisy_folder = os.path.join(base_noisy_dir, f"CATHLAB Noisy ({noise_level})")
    denoised_folder = os.path.join(base_denoised_dir, f"CATHLAB Noisy ({noise_level})")

    psnr_results = []
    psnr_noisy_list = []
    psnr_denoised_list = []

    for filename in files_to_process:
        normal_path = os.path.join(base_original_dir, filename + ".dcm")
        noisy_path = os.path.join(noisy_folder, filename + ".dcm")
        denoised_path = os.path.join(denoised_folder, filename + ".dcm")

        # Ensure all files exist
        if not (os.path.exists(normal_path) and os.path.exists(noisy_path) and os.path.exists(denoised_path)):
            print(f"Skipping {filename} for noise level {noise_level}: Missing file(s).")
            continue

        # Load DICOM images
        normal_dcm = pydicom.dcmread(normal_path)
        noisy_dcm = pydicom.dcmread(noisy_path)
        denoised_dcm = pydicom.dcmread(denoised_path)

        # Convert pixel data to float format
        normal_img = img_as_float(np.squeeze(normal_dcm.pixel_array))
        noisy_img = img_as_float(np.squeeze(noisy_dcm.pixel_array))
        denoised_img = img_as_float(np.squeeze(denoised_dcm.pixel_array))

        # Compute PSNR
        psnr_noisy = peak_signal_noise_ratio(normal_img, noisy_img)
        psnr_denoised = peak_signal_noise_ratio(normal_img, denoised_img)

        # Store the results
        psnr_results.append((filename, psnr_noisy, psnr_denoised))
        psnr_noisy_list.append(psnr_noisy)
        psnr_denoised_list.append(psnr_denoised)

    # Compute mean and standard deviation
    if psnr_noisy_list and psnr_denoised_list:
        mean_psnr_noisy = np.mean(psnr_noisy_list)
        std_psnr_noisy = np.std(psnr_noisy_list)
        mean_psnr_denoised = np.mean(psnr_denoised_list)
        std_psnr_denoised = np.std(psnr_denoised_list)

        # Store results in dictionary
        psnr_summary[noise_level] = {
            "mean_noisy": mean_psnr_noisy,
            "std_noisy": std_psnr_noisy,
            "mean_denoised": mean_psnr_denoised,
            "std_denoised": std_psnr_denoised,
            "results": psnr_results
        }

    # Print results for the current noise level
    print("\nPSNR Results:")
    print(f"{'Filename':<10} {'PSNR (Noisy)':<15} {'PSNR (Denoised)':<15}")
    print("=" * 40)
    for filename, psnr_noisy, psnr_denoised in psnr_results:
        print(f"{filename:<10} {psnr_noisy:<15.2f} {psnr_denoised:<15.2f}")

    # Print mean and standard deviation
    print("=" * 40)
    print(f"{'Mean':<10} {mean_psnr_noisy:<15.2f} {mean_psnr_denoised:<15.2f}")
    print(f"{'Std Dev':<10} {std_psnr_noisy:<15.2f} {std_psnr_denoised:<15.2f}")

# Final summary across all noise levels
print("\nFinal Summary of All Noise Levels:")
print(f"{'Noise Level':<10} {'Mean PSNR (Noisy)':<20} {'Mean PSNR (Denoised)':<20}")
print("=" * 60)
for noise_level, values in psnr_summary.items():
    print(f"{noise_level:<10} {values['mean_noisy']:<20.2f} {values['mean_denoised']:<20.2f}")

print("\nProcessing complete.")

In [ ]:
import matplotlib.pyplot as plt

# Extract data for plotting
noise_levels = sorted(psnr_summary.keys())  # Ensure noise levels are in ascending order
mean_psnr_noisy = [psnr_summary[nl]["mean_noisy"] for nl in noise_levels]
mean_psnr_denoised = [psnr_summary[nl]["mean_denoised"] for nl in noise_levels]

# Plot PSNR values
plt.figure(figsize=(10, 6))
plt.plot(noise_levels, mean_psnr_noisy, marker='o', linestyle='-', color='r', label='Noisy Images')
plt.plot(noise_levels, mean_psnr_denoised, marker='s', linestyle='-', color='b', label='Denoised Images')

# Labels and title
plt.xlabel("Poisson Noise Standard Deviation")
plt.ylabel("Mean PSNR (dB)")
plt.title("PSNR Analysis of Noisy vs. Denoised Images")
plt.legend()
plt.grid(True)